# Playbook Explorer

This notebook is intended to be a live example of how to work with SysML v2 models at analysis-time. For these purposes, the following terms are introduced:
* An *interpretation* is the mapping of user model symbols (the "M1 model" in OMG-speak) into semantically-correct symbols that represent real world objects meant to conform to the model (the "M0" in OMG-speak). Interpretation semantics are inspired by https://www.w3.org/TR/owl2-direct-semantics/ and are mostly similar.
* A *sequence* for an interpretation contains *atoms* or *instances* that match to real world things. Reading a sequence from left to right provides a set of nested contexts for the atoms that is important to the interpretation. For example [Rocket#0, LS#3] is a 2-sequence to describe facts around the LS#3 atom when it is considered in context for Rocket#0. This is an important idea for the SysML time and occurrence model where one may want to see how values change under different conditions.

This is a notebook that walks through the random interpretation generator to help developers working on their own interpreters.

## Example Model

The model that is used for this example is a very simple parts model.

    package 'Simple Parts Model' {
    import Base::Anything;
    package 'Fake Library' {
        part def Part;
        port def Port;
        connection def Connection;
    }
    
    part 'Power Group' : 'Fake Library'::Part {
        part 'Power Source' : 'Fake Library'::Part [2] {
            port 'Power Out' : 'Fake Library'::Port;
        }
        part 'Power User' : 'Fake Library'::Part [4] {
            port 'Power In' : 'Fake Library'::Port;
        }
        
        connection powerToUser : 'Fake Library'::Connection [4] connect 'Power Source'::'Power Out' to 'Power User'::'Power In'; 
    }
    
}

## Imports

Import key modules, functions, and classes from the PyMBE library:

In [ ]:
from pathlib import Path
import networkx as nx

import pymbe.api as pm

from pymbe.client import SysML2Client
from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.interpretation import repack_instance_dictionaries
from pymbe.interpretation.interp_playbooks import (
    build_expression_sequence_templates,
    build_sequence_templates,
    random_generator_playbook,
)
from pymbe.interpretation.results import *
from pymbe.label import get_label_for_id
from pymbe.query.metamodel_navigator import feature_multiplicity
from pymbe.query.query import (
    roll_up_multiplicity,
    roll_up_upper_multiplicity,
    roll_up_multiplicity_for_type,
    get_types_for_feature,
    get_features_typed_by_type,
)
from pymbe.local.stablization import build_stable_id_lookups

## Client Setup

The example here uses a local copy of the JSON file obtained by a GET operation on the SysML v2 API at:

http://sysml2-sst.intercax.com:9000/projects/af60242b-c684-4b55-b628-aeb303bde9a7/commits/a6927839-3d52-4352-811c-d50cd21186e6/elements?page[size]=5000

Create the client and load local data.

In [ ]:
helper_client = SysML2Client()

simple_parts_file = Path(pm.__file__).parent / "../../tests/fixtures/Simple Parts Model.json"

helper_client._load_from_file(simple_parts_file)

Create a graph representation of the model and load it into memory.

In [ ]:
lpg = SysML2LabeledPropertyGraph()
lpg.model = helper_client.model

This is just a helper to make abbreviations more legible.

In [ ]:
name_hints = {}

Create an interpretation of the Kerbal model using the random generator playbook. In general, this randomly selects:
- The ratios of partitioning abstract classifier sequence sets into concrete sets. For example, one draw may choose 2 liquid stages and 3 solids.
- The number of sequences to create for a given feature multiplicity. For example, draw 2 for a 0..8 engines : Liquid Engine PartUsage.

The playbook also attempts to make sequences created obey the Subsetting relationship (elements marked with subsets in M1 model should have their interpretation sequences entirely included within the interpretation sequences of the superset).

## Stable Names for Identifiers

Calculate unique names for items in the model through a combination of qualified names and the generation of signatures for different steps in the expression trees.

In [ ]:
SIMPLE_MODEL = "Model::Simple Parts Model::"

In [ ]:
name_to_id_lookup = build_stable_id_lookups(lpg)[1]

[
    name
    for name in name_to_id_lookup
    if "Connections::Connection <" in name
]

In [ ]:
connection_id = name_to_id_lookup["Model::Connections::Connection <<ConnectionDefinition>>"]
part_id = name_to_id_lookup["Model::Parts::Part <<PartDefinition>>"]
port_id = name_to_id_lookup["Model::Ports::Port <<PortDefinition>>"]

power_source_id = name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::Power Source: Part <<PartUsage>>"]
power_user_id = name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::Power User: Part <<PartUsage>>"]
power_in_id = name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::Power User: Part::Power In: Port <<PortUsage>>"]
power_out_id = name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::Power Source: Part::Power Out: Port <<PortUsage>>"]
connect_use_id = name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::powerToUser: Connection <<ConnectionUsage>>"]
power_group_id = name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part <<PartUsage>>"]

## M1 Model Exploration

Look at the properties of M1 and do some queries to check on typing and multiplicity.

### Inspect Featuring Typing and Multiplicities

Inspect the M1 model to see how many of a given type need to be instantiated to populate sequences.

In [ ]:
pprint_single_id_list(get_types_for_feature(lpg, power_source_id), lpg.model)

In [ ]:
pprint_single_id_list(get_features_typed_by_type(lpg, part_id), lpg.model)

In [ ]:
roll_up_multiplicity_for_type(lpg, lpg.model.elements[connection_id], 'upper')

In [ ]:
roll_up_multiplicity(lpg, lpg.model.elements[power_in_id], 'upper')

In [ ]:
roll_up_multiplicity(lpg, lpg.model.elements[power_user_id], 'upper')

In [ ]:
roll_up_multiplicity(lpg, lpg.model.elements[connect_use_id], 'upper')

In [ ]:
feature_multiplicity(lpg.model.elements[connection_id], "upper")

### Expanded Banded Graph

The graph below is used to compute multiplicities for features by taking into account the multiplicities of nesting features and inherited types.

In [ ]:
banded_featuring_graph = lpg.get_projection("Expanded Banded")
banded_roots = [
        node
        for node in banded_featuring_graph.nodes
        if banded_featuring_graph.out_degree(node) < 1
    ]
pprint_single_id_list(banded_roots, lpg.model)

In [ ]:
pprint_double_id_list(build_sequence_templates(lpg), lpg.model)

In [ ]:
m0_interpretation = random_generator_playbook(
    lpg=lpg,
    name_hints=name_hints,
    filtered_feat_packages=[lpg.model.ownedElement["Simple Parts Model"]],
)

In [ ]:
for element in lpg.model.elements.values():
    if element._data.get("type"):
        break
        
element, element.type

## Interpretation Results

The following cells are a series of displays of relevant features in the interpretation.

Show all interpretation sequence sets (limited to length of 5).

In [ ]:
for print_line in pprint_interpretation(m0_interpretation, lpg.model):
    print(print_line)

Interpretations of the ports:

In [ ]:
m0_interpretation[power_in_id]

In [ ]:
m0_interpretation[power_out_id]

Interpretation of the connections inside:

In [ ]:
m0_interpretation[connect_use_id]

Interpretation of the connection ends:

In [ ]:
m0_interpretation[lpg.model.elements[connect_use_id].connectorEnd[0]._id]

In [ ]:
m0_interpretation[lpg.model.elements[connect_use_id].connectorEnd[1]._id]

## Drawing Interpretations

This section walks through how to find the needed hints for drawing interpretations.

In [ ]:
repacked = repack_instance_dictionaries(m0_interpretation, lpg.model)
repacked

In [ ]:
connection_usage_entry = [
    entry
    for entry in repacked
    if entry.base._metatype == "ConnectionUsage"
][0]

In [ ]:
list(connection_usage_entry.value)[0].owning_entry.draw_kind

In [ ]:
[list_item.get_line_ends() for list_item in list(connection_usage_entry.value)]

In [ ]:
connection_usage_entry.base

In [ ]:
for entry in repacked:
    kind = list(entry.value)[0].owning_entry.draw_kind
    if not kind:
        continue
    print(kind)
    print(entry)